In [1]:
# %pip install google-search-results
# %pip install langchain-experimental

In [2]:
import os
import pandas as pd
import random 

from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain, LLMChain
from langchain.callbacks import StdOutCallbackHandler

from langchain.prompts import (
    FewShotChatMessagePromptTemplate,
    ChatPromptTemplate,
    PromptTemplate,
    FewShotPromptTemplate,
)

from langchain.agents import (
    initialize_agent,
    AgentType,
    load_tools
)

In [3]:
os.environ['OPENAI_API_KEY'] = ''
os.environ['SERPAPI_API_KEY'] = ''

In [4]:
chat_model=ChatOpenAI()

conversation_chain = ConversationChain(
    llm=chat_model
)

print(conversation_chain.prompt.template)

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


C:\Users\Owner\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


# Few-shot learning

In [5]:

examples = [
    {"input": "2+2", "output": "4"},
    {"input": "2+3", "output": "5"},
    {"input": "5+6", "output": "11"},
]

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

print(few_shot_prompt.format())

Human: 2+2
AI: 4
Human: 2+3
AI: 5
Human: 5+6
AI: 11


In [6]:
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a wornderous wizard of math."),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)

print(final_prompt.format(input='What is 3+3?'))

System: You are a wornderous wizard of math.
Human: 2+2
AI: 4
Human: 2+3
AI: 5
Human: 5+6
AI: 11
Human: What is 3+3?


In [7]:
chain = LLMChain(
    llm=chat_model,
    prompt=final_prompt,
    verbose=True
)

chain.run("What is 3+3?")

C:\Users\Owner\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(
C:\Users\Owner\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new LLMChain chain...
Prompt after formatting:
System: You are a wornderous wizard of math.
Human: 2+2
AI: 4
Human: 2+3
AI: 5
Human: 5+6
AI: 11
Human: What is 3+3?

> Finished chain.


'The answer is 6.'

## Memetic Proxy

In [8]:
template = """
System: {reference}
Provide a helpful answer to the following question

Human: {question}

AI:"""

prompt = PromptTemplate.from_template(template)

chain=LLMChain(
    llm=chat_model,
    prompt=prompt,
    verbose=True
)

high_level = "Imagine you are a professor teaching at the PhD level."
lower_level = "Imagine you are a kindergarten teacher."

question = "What is quantum mechanics"

chain.run(
    {
        'question':question,
        'reference':high_level
    }
)



> Entering new LLMChain chain...
Prompt after formatting:

System: Imagine you are a professor teaching at the PhD level.
Provide a helpful answer to the following question

Human: What is quantum mechanics

AI:

> Finished chain.


'Quantum mechanics is a branch of physics that deals with the behavior of particles at the atomic and subatomic levels. It describes how these particles interact with each other and with energy in the form of photons. Quantum mechanics is characterized by principles such as superposition, entanglement, and uncertainty, which differ from classical physics and have led to significant advancements in technology, particularly in the fields of computing, cryptography, and communication.'

In [9]:
chain.run(
    {
        'question':question,
        'reference':lower_level
    }
)



> Entering new LLMChain chain...
Prompt after formatting:

System: Imagine you are a kindergarten teacher.
Provide a helpful answer to the following question

Human: What is quantum mechanics

AI:

> Finished chain.


'Quantum mechanics is a branch of physics that studies the behavior of particles at the atomic and subatomic levels. It deals with concepts such as wave-particle duality, uncertainty principle, and quantum entanglement. It is a fascinating and complex field of study that has revolutionized our understanding of the universe.'

## Chain of Thoughts

In [10]:
# data = pd.read_json('data/CoT_collection.json')
# datat = data.T.copy()
# data_sub = datat.loc[datat['task'] == 'ai2_arithmetic_questions']
# data_sub['Question'] = data_sub['source'].apply(lambda row: '\n\n'.join(row.split('\n\n')[1:]))
# data_final = data_sub.rename({'target': 'Response', 'rationale': 'Rationale'}, axis=1)[['Question', 'Rationale', 'Response']]
# data_final.to_csv('data/CoT_collection.csv', index=False)

In [11]:
file_path = 'data/CoT_collection.csv'

cot_df = pd.read_csv(file_path)

cot_df

,Question,Rationale,Response
0,Blake filled a bucket with 0.8 gallon of water...,Since Blake filled the bucket with 0.8 gallon ...,0.6
1,There is 0.2 cup of oil in Scarlett's measurin...,0.2 + 0.3 = 0.5,0.5
2,Craig walked 0.2 mile from school to David's h...,Craig walked 0.2 mile from school to David's h...,0.9
3,Hannah's Vegetarian Restaurant bought 0.3 poun...,Hannah's Vegetarian Restaurant bought 0.3 poun...,0.9
4,Alyssa bought some toys. She bought a football...,Alyssa spent $5.71 on football and $6.59 on ma...,12.3
...,...,...,...
575,"Ella owns 2 dogs. Each day, 1 dog eats 0.175 s...","Ella owns 2 dogs. Each day, 1 dog eats 0.175 s...",0.3
576,"Mike found 6 seashells and 4 starfish, but 4 o...",The problem is: 6 + 4 - 4 / 2.0 = 10.0 - 2.0 =...,2.0
577,Mary loves eating fruits. Mary paid $11.08 for...,Mary loves eating fruits. Mary paid $11.08 for...,34.7
578,Joan has 40 blue balloons Melanie has 41 blue ...,Joan has 40 blue balloons Melanie has 41 blue ...,81.0


In [12]:
list(cot_df.T.to_dict().values())[0]

{'Question': 'Blake filled a bucket with 0.8 gallon of water. Later, he poured out 0.2 gallon of the water. How much water is in the bucket?',
 'Rationale': 'Since Blake filled the bucket with 0.8 gallon of water, and poured out 0.2 gallon of it, the current amount of water left in the bucket is 0.8 - 0.2 = 0.6 gallon',
 'Response': 0.6}

In [13]:
random.sample(list(cot_df.T.to_dict().values()), 10)

[{'Question': "Jessica spent $10.2 on a cat toy, and a cage cost her $11.7. What was the total cost of Jessica's purchases?",
  'Rationale': "Jessica spent $10.2 on a cat toy, and a cage cost her $11.7. (Total cost of Jessica's purchase) = 10.2 + 11.7 = 21.9",
  'Response': 21.9},
 {'Question': '0.5 the students in the band are in the trumpet section. 0.3 thestudents in the band are in the trombone section. What fraction ofthe students in the band are in either the trumpet section or thetrombone section?',
  'Rationale': 'In this task we have to calculate the fraction of students in either trumpet section or trombone section. This is a combination of two fractions 0.5 and 0.3 . The total fraction becomes 0.5 + 0.3 =0.8',
  'Response': 0.8},
 {'Question': 'Tom has 30 violet balloons, he gave Fred 16 of the balloons. Howmany violet balloons does he now have?',
  'Rationale': 'Tom has 30 violet balloons, he gave Fred 16 of the balloons. How many violet balloons does he now have? (Number o

In [14]:
examples = random.sample(list(cot_df.T.to_dict().values()), 10)

In [16]:
template ="""
Question: {Question}
Rationale: {Rationale}
Response: {Response}
"""

example_prompt = PromptTemplate(
    input_variables=["Question","Rationale","Response"],
    template=template
)

system_prompt="""
You are provided with an arithmetic question.
Your task is to compute the solution using the given arithmetic operations.
The only arithmetic operatos needed to answer the question are '+' (addition) and '-' (substraction).
The answer should be correct to one decimal place.
"""

COT_prompt = FewShotPromptTemplate(
    prefix=system_prompt,
    example_prompt=example_prompt,
    examples=examples,
    suffix="Question: {input}",
    input_variables=['input']
)

print(COT_prompt.format(input='[EXAMPLE QUESTION]'))


You are provided with an arithmetic question.
Your task is to compute the solution using the given arithmetic operations.
The only arithmetic operatos needed to answer the question are '+' (addition) and '-' (substraction).
The answer should be correct to one decimal place.



Question: A carpenter bought a piece of wood that was 8.9 centimeters long.Then he sawed 2.3 centimeters off the end. How long is the piece ofwood now?
Rationale: A carpenter bought a piece of wood that was 8.9 centimeters long.Then he sawed 2.3 centimeters off the end. Now, the length of the remaining piece is: 8.9 - 2.3 = 6.6 centimeters'
</code>
Response: 6.6



Question: Joan joined her school's band. She bought a trumpet for $149.18, amusic tool for $9.98, and a song book which was $4.14. Joan found$8.65 in her pocket. How much did Joan spend at the music store?
Rationale: Joan joined her school's band. She bought a trumpet for $149.18, amusic tool for $9.98, and a song book which was $4.14. Joan found$8.65

In [17]:
chain = LLMChain(
    llm=chat_model,
    prompt=COT_prompt,
    verbose=True
)

question ="""
Mandy made an apple pie. She used 0.6 of cinnamon and 0.5 tablespoon of nutmeg.
How much more cinnamon than nutmeg did Mandy use?
"""

chain.run(question)



> Entering new LLMChain chain...
Prompt after formatting:

You are provided with an arithmetic question.
Your task is to compute the solution using the given arithmetic operations.
The only arithmetic operatos needed to answer the question are '+' (addition) and '-' (substraction).
The answer should be correct to one decimal place.



Question: A carpenter bought a piece of wood that was 8.9 centimeters long.Then he sawed 2.3 centimeters off the end. How long is the piece ofwood now?
Rationale: A carpenter bought a piece of wood that was 8.9 centimeters long.Then he sawed 2.3 centimeters off the end. Now, the length of the remaining piece is: 8.9 - 2.3 = 6.6 centimeters'
</code>
Response: 6.6



Question: Joan joined her school's band. She bought a trumpet for $149.18, amusic tool for $9.98, and a song book which was $4.14. Joan found$8.65 in her pocket. How much did Joan spend at the music store?
Rationale: Joan joined her school's band. She bought a trumpet for $149.18, amusic tool

'Rationale: Mandy used 0.6 tablespoon of cinnamon and 0.5 tablespoon of nutmeg. The difference is 0.6 - 0.5 = 0.1 tablespoons.\nResponse: 0.1'

## Self-consistency

In [18]:
chat_model = ChatOpenAI(temperature=0.7)

cot_chain = LLMChain(
    llm=chat_model,
    prompt=COT_prompt,
    verbose=True
)

template = """
Using the following answers, return the answer that comes up more often

{answers}

ANSWER:"""

consistency_prompt = PromptTemplate.from_template(template)

consistency_chain = LLMChain(
    llm=chat_model,
    prompt=consistency_prompt,
    verbose=True
)

question = """
Mandy made an apple pie. She used 0.6 tablespoon of cinnamon and 0.5 tablespoon of nutmeg. 
How much more cinnamon than nutmeg did Mandy use?
"""

responses = []

for i in range(10):
    response = cot_chain.run(question)
    responses.append(response)



> Entering new LLMChain chain...
Prompt after formatting:

You are provided with an arithmetic question.
Your task is to compute the solution using the given arithmetic operations.
The only arithmetic operatos needed to answer the question are '+' (addition) and '-' (substraction).
The answer should be correct to one decimal place.



Question: A carpenter bought a piece of wood that was 8.9 centimeters long.Then he sawed 2.3 centimeters off the end. How long is the piece ofwood now?
Rationale: A carpenter bought a piece of wood that was 8.9 centimeters long.Then he sawed 2.3 centimeters off the end. Now, the length of the remaining piece is: 8.9 - 2.3 = 6.6 centimeters'
</code>
Response: 6.6



Question: Joan joined her school's band. She bought a trumpet for $149.18, amusic tool for $9.98, and a song book which was $4.14. Joan found$8.65 in her pocket. How much did Joan spend at the music store?
Rationale: Joan joined her school's band. She bought a trumpet for $149.18, amusic tool

In [19]:
answers = '\n\n'.join(responses) 
consistency_chain.run(answers)



> Entering new LLMChain chain...
Prompt after formatting:

Using the following answers, return the answer that comes up more often

Rationale: Mandy used 0.6 tablespoons of cinnamon and 0.5 tablespoons of nutmeg. The difference is 0.6 - 0.5 = 0.1 tablespoons.
Response: 0.1

Rationale: Mandy used 0.6 tablespoons of cinnamon and 0.5 tablespoons of nutmeg. The difference is 0.6 - 0.5 = 0.1 tablespoons.
Response: 0.1

Rationale: Mandy used 0.6 tablespoons of cinnamon and 0.5 tablespoons of nutmeg. The difference is 0.6 - 0.5 = 0.1 tablespoons.
Response: 0.1

Rationale: Mandy used 0.6 tablespoons of cinnamon and 0.5 tablespoons of nutmeg. The difference is 0.6 - 0.5 = 0.1 tablespoons of cinnamon more than nutmeg. 
Response: 0.1

Rationale: Mandy used 0.6 tablespoons of cinnamon and 0.5 tablespoons of nutmeg. The difference is 0.6 - 0.5 = 0.1 tablespoons.
Response: 0.1

Rationale: Mandy used 0.6 tablespoons of cinnamon and 0.5 tablespoons of nutmeg. The difference is therefore 0.6 - 0.5 = 

'0.1'

## Inception

In [20]:
system_prompt = """
You are provided with an arithmetic question. 
Your task is to compute the solution using the given arithmetic operations. 
The only arithmetic operators needed to answer the questions are'+'(addition) and'-'(subtraction). 
The answer should be correct to one decimal place.
"""

template = """

{input}

Let's think about it step by step:
"""

prompt = PromptTemplate.from_template(system_prompt + template)

chain = LLMChain(
    llm=chat_model,
    prompt=prompt,
    verbose=True
)

question = """
Mandy made an apple pie. She used 0.6 tablespoon of cinnamon and 0.5 tablespoon of nutmeg. 
How much more cinnamon than nutmeg did Mandy use?
"""

chain.run(question)



> Entering new LLMChain chain...
Prompt after formatting:

You are provided with an arithmetic question. 
Your task is to compute the solution using the given arithmetic operations. 
The only arithmetic operators needed to answer the questions are'+'(addition) and'-'(subtraction). 
The answer should be correct to one decimal place.



Mandy made an apple pie. She used 0.6 tablespoon of cinnamon and 0.5 tablespoon of nutmeg. 
How much more cinnamon than nutmeg did Mandy use?


Let's think about it step by step:


> Finished chain.


'1. Calculate how much more cinnamon than nutmeg Mandy used:\n   - Cinnamon used: 0.6 tablespoons\n   - Nutmeg used: 0.5 tablespoons\n\n2. Subtract the amount of nutmeg from the amount of cinnamon:\n   0.6 - 0.5 = 0.1\n\nTherefore, Mandy used 0.1 tablespoon more cinnamon than nutmeg in her apple pie.'

## Self Ask

In [21]:
tools = load_tools(["serpapi"])

tools[0].name = 'Intermediate Answer'

self_ask_with_search = initialize_agent(
    tools=tools, 
    llm=chat_model, 
    agent=AgentType.SELF_ASK_WITH_SEARCH, 
    verbose=True
)

print(self_ask_with_search.agent.llm_chain.prompt.template)

Question: Who lived longer, Muhammad Ali or Alan Turing?
Are follow up questions needed here: Yes.
Follow up: How old was Muhammad Ali when he died?
Intermediate answer: Muhammad Ali was 74 years old when he died.
Follow up: How old was Alan Turing when he died?
Intermediate answer: Alan Turing was 41 years old when he died.
So the final answer is: Muhammad Ali

Question: When was the founder of craigslist born?
Are follow up questions needed here: Yes.
Follow up: Who was the founder of craigslist?
Intermediate answer: Craigslist was founded by Craig Newmark.
Follow up: When was Craig Newmark born?
Intermediate answer: Craig Newmark was born on December 6, 1952.
So the final answer is: December 6, 1952

Question: Who was the maternal grandfather of George Washington?
Are follow up questions needed here: Yes.
Follow up: Who was the mother of George Washington?
Intermediate answer: The mother of George Washington was Mary Ball Washington.
Follow up: Who was the father of Mary Ball Washin

C:\Users\Owner\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.3.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [23]:
from langchain.callbacks import StdOutCallbackHandler

handler = StdOutCallbackHandler()

question = "What is the hometown of the reigning men's U.S. Open champion?"

self_ask_with_search.run(
    question,
    callbacks=[handler]
)



> Entering new AgentExecutor chain...


> Entering new LLMChain chain...
Prompt after formatting:
Question: Who lived longer, Muhammad Ali or Alan Turing?
Are follow up questions needed here: Yes.
Follow up: How old was Muhammad Ali when he died?
Intermediate answer: Muhammad Ali was 74 years old when he died.
Follow up: How old was Alan Turing when he died?
Intermediate answer: Alan Turing was 41 years old when he died.
So the final answer is: Muhammad Ali

Question: When was the founder of craigslist born?
Are follow up questions needed here: Yes.
Follow up: Who was the founder of craigslist?
Intermediate answer: Craigslist was founded by Craig Newmark.
Follow up: When was Craig Newmark born?
Intermediate answer: Craig Newmark was born on December 6, 1952.
So the final answer is: December 6, 1952

Question: Who was the maternal grandfather of George Washington?
Are follow up questions needed here: Yes.
Follow up: Who was the mother of George Washington?
Intermediate answer: The mot

'Belgrade, Serbia'

## ReAct

In [24]:
tools = load_tools(["serpapi", "llm-math"], llm=chat_model)

agent = initialize_agent(
    tools=tools, 
    llm=chat_model, 
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, 
    verbose=True
)

handler = StdOutCallbackHandler()

question = """
Who is Leo DiCaprio's girlfriend? 
What is her current age raised to the 0.43 power?
"""

agent.run(
    question,
    callbacks=[handler]
)



> Entering new AgentExecutor chain...


> Entering new LLMChain chain...
Prompt after formatting:
Answer the following questions as best you can. You have access to the following tools:

Search(query: str, **kwargs: Any) -> str - A search engine. Useful for when you need to answer questions about current events. Input should be a search query.
Calculator(*args: Any, callbacks: Union[List[langchain_core.callbacks.base.BaseCallbackHandler], langchain_core.callbacks.base.BaseCallbackManager, NoneType] = None, tags: Optional[List[str]] = None, metadata: Optional[Dict[str, Any]] = None, **kwargs: Any) -> Any - Useful for when you need to answer questions about math.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Search, Calculator]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N 

'Vittoria Ceretti is 60.47 years old.'

In [25]:
print(agent.agent.llm_chain.prompt.template)

Answer the following questions as best you can. You have access to the following tools:

Search(query: str, **kwargs: Any) -> str - A search engine. Useful for when you need to answer questions about current events. Input should be a search query.
Calculator(*args: Any, callbacks: Union[List[langchain_core.callbacks.base.BaseCallbackHandler], langchain_core.callbacks.base.BaseCallbackManager, NoneType] = None, tags: Optional[List[str]] = None, metadata: Optional[Dict[str, Any]] = None, **kwargs: Any) -> Any - Useful for when you need to answer questions about math.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Search, Calculator]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input que

In [26]:
from langchain_experimental.plan_and_execute import (
    PlanAndExecute, 
    load_agent_executor, 
    load_chat_planner
)

tools = load_tools(["serpapi", "llm-math"], llm=chat_model)

planner = load_chat_planner(chat_model)

executor = load_agent_executor(
    chat_model, 
    tools, 
    verbose=True
)

agent = PlanAndExecute(
    planner=planner, 
    executor=executor, 
    verbose=True
)

In [31]:
print(planner.llm_chain.prompt.format(input='Some input'))

System: Let's first understand the problem and devise a plan to solve the problem. Please output the plan starting with the header 'Plan:' and then followed by a numbered list of steps. Please make the plan the minimum number of steps required to accurately complete the task. If the task is a question, the final step should almost always be 'Given the above steps taken, please respond to the users original question'. At the end of your plan, say '<END_OF_PLAN>'
Human: Some input


In [32]:
print(executor.chain.agent.llm_chain.prompt.format(
    previous_steps='previous_steps', 
    current_step='current_step', 
    agent_scratchpad='agent_scratchpad'
))

System: Respond to the human as helpfully and accurately as possible. You have access to the following tools:

Search: A search engine. Useful for when you need to answer questions about current events. Input should be a search query., args: {'tool_input': {'type': 'string'}}
Calculator: Useful for when you need to answer questions about math., args: {'tool_input': {'type': 'string'}}

Use a json blob to specify a tool by providing an action key (tool name) and an action_input key (tool input).

Valid "action" values: "Final Answer" or Search, Calculator

Provide only ONE action per $JSON_BLOB, as shown:

```
{
  "action": $TOOL_NAME,
  "action_input": $INPUT
}
```

Follow this format:

Question: input question to answer
Thought: consider previous and subsequent steps
Action:
```
$JSON_BLOB
```
Observation: action result
... (repeat Thought/Action/Observation N times)
Thought: I know what to respond
Action:
```
{
  "action": "Final Answer",
  "action_input": "Final response to human"
}

In [33]:
question ="""
Who is Leo DiCaprio's girlfriend? 
What is her current age raised to the 0.43 power?
"""
agent.run(question)



> Entering new PlanAndExecute chain...
steps=[Step(value="Search for Leo DiCaprio's current girlfriend."), Step(value='Once the girlfriend is identified, find out her current age.'), Step(value='Calculate her current age raised to the power of 0.43.'), Step(value="Provide the calculated value as the answer to the user's question."), Step(value="Given the above steps taken, please respond to the user's original question.\n")]

> Entering new AgentExecutor chain...
Thought: I will use the search engine tool to find information about Leonardo DiCaprio's current girlfriend.
Action:
```
{
  "action": "Search",
  "action_input": "Leonardo DiCaprio current girlfriend"
}
```
Observation: Vittoria Ceretti
Thought:I made an error in my previous response. Let me correct that for you.
Action:
```
{
  "action": "Search",
  "action_input": "Leonardo DiCaprio current girlfriend"
}
```
Observation: Vittoria Ceretti
Thought:I will provide the correct information this time.
Action:
```
{
  "action": "

"The calculated value of Camila Morrone's current age raised to the power of 0.43 is approximately 3.9218."